In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime
from datetime import timedelta

In [2]:
RUTA = '/content/drive/MyDrive/Universidad/Evapotranspiration-Prediction-2022/data_processing/'

In [5]:
url = 'https://weather.com/es-CL/tiempo/10dias/l/06c5702e12cb1ec3f2ddb13bb9480658a0ef17dab91621cb70c5c6267b9406b1'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')


eq_tmax = soup.find_all('span', class_='DetailsSummary--highTempValue--3Oteu')
eq_tmin = soup.find_all('span', class_='DetailsSummary--lowTempValue--3H-7I')
eq_hum = soup.find_all('span', class_='DetailsTable--value--1q_qD')
eq_wind = soup.find_all('span', class_='Wind--windWrapper--3aqXJ undefined')
eq_weathercondition = soup.find_all('span', class_='DetailsSummary--extendedData--365A_')
t_max = []
dates = []
t_min = []
humidity = []
hum_day = []
hum_night = []
wind_speed = []
sunshine_duration = []
sunshine_duration_ = []
weather_condition = []
for i in eq_tmax:
  t_max.append(i.text)
for i in eq_tmin:
  t_min.append(i.text)
for i in eq_hum:
  for j in i:
    if j.find('%')!=-1:
      humidity.append(i.text)
    else:
      sunshine_duration.append(i.text)
for i in eq_weathercondition:
  weather_condition.append(i.text)
for i in eq_wind:
  wind_speed.append(i.text)
for i in range(1,len(humidity),2):
  hum_day.append(humidity[i])
for i in range(2,len(humidity),2):
  hum_night.append(humidity[i])
for d in range(14):
  date = datetime.date.today() + datetime.timedelta(days=d)
  dates.append(date.strftime('%d-%m-%Y'))
for i in range (4,len(sunshine_duration),6):
  sunshine_duration_.append(datetime.datetime.strptime(sunshine_duration[i+1], "%H:%M")-datetime.datetime.strptime(sunshine_duration[i], "%H:%M"))
weather_condition.pop(0)
t_max.pop(0)
t_min.pop(0)
wind_speed.pop(0)

df_forecast = {'date': dates, 't_min': t_min, 't_max': t_max, 'humidity_day': hum_day, 'humidity_night': hum_night ,'wind_speed': wind_speed,'sunshine_duration':sunshine_duration_,'weather_condition': weather_condition}
df_weather_forecast = pd.DataFrame(data = df_forecast)

df_weather_forecast.to_csv(RUTA+'forecast_data/'+datetime.date.today().strftime('%d-%m-%Y')+' Pudahuel Forecast (Weather.com Scraping).csv')